In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

C:\Users\AVasilev\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

In [10]:
data = pd.read_csv('./data/retail_train.csv')

item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [11]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [12]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [13]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [14]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [15]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [16]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

## 1. Filter items

In [17]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [18]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\AVasilev\Jupyter Notebook\Recommendations-systems\utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [19]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [20]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [21]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [22]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
user_feat.shape

(2497, 7)

In [24]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


In [25]:
item_feat.shape

(5001, 6)

## Encoding features

In [26]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [27]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Init model

In [21]:
model = LightFM(no_components=40,
                loss='warp', # "logistic","bpr"
                learning_rate=0.01, 
                item_alpha=0.4,
                user_alpha=0.1, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)

## Train

In [22]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True) 

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [02:25<00:00,  7.28s/it]


# Getting embeddings

## вектора по пользователям

In [23]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [24]:
user_emb[0].shape # biases

(2497,)

In [25]:
user_emb[1].shape # users vectors

(2497, 40)

## вектора по товарам

In [26]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [27]:
item_emb[0].shape # biases

(5001,)

In [28]:
item_emb[1].shape # items vectors

(5001, 40)

# Evaluation -> Train precision

In [29]:
# мы можем использовать встроенные метрики lightFM
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

print(f"Train precision {train_precision}")

Train precision 0.2941129505634308


# Predict

In [28]:
# подготавливаемм id для юзеров и товаров в порядке пар user-item
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [31]:
# модель возвращает меру/скор похожести между соответствующим пользователем и товаром
predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)

In [32]:
# добавляем наш полученный скор в трейн датафрейм
data_train_filtered['score'] = predictions

In [34]:
# создаем предикт датафрейм в формате списка това
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()

In [35]:
# объединяем предикт и тест датасет для подсчета precision
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

In [36]:
df_result_for_metrics.head()

,user_id,actual,item_id
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1029743, 986912, 6034857, 12810393, 952163, 1..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1127831, 12810393, 965267, 874972, 8..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1070820, 1029743, 1126899, 986912, 1127831, 9..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1029743, 1126899, 1106523, 1127831, 12810393,..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1106523, 1070820, 1029743, 986912, 916122, 60..."


### Test with custom precision func

In [37]:
precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

Precision: 0.1462397372742181


# Домашнее задание

#### 1) Прочитать статьи про BPR, WARP loss

#### 2) Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией

In [38]:
# !pip install swifter

In [31]:
# В статье на towardsdatascience.com писали, что swifter выполняет apply быстрее
import swifter
from itertools import product

In [37]:
# Списки параметров для перебора
loss_func = ['bpr']
learning_rates = [0.1, 0.05, 0.01]
no_components_list = [20, 40, 60]
n_list = [10, 15, 20]
item_alphas = [0.2, 0.4, 0.6]
user_alphas = [0.1, 0.2, 0.3]

In [38]:
result_dict = {
    'precision_at_k': [],
    'loss': [],
    'learning_rate': [],
    'no_components': [],
    'n': [],
    'item_alpha': [],
    'user_alpha': []
}

In [39]:
def get_precision(df, data_train, sparse_user_item_matrix, user_item_mx, 
                  user_feat_lfm, item_feat_lfm, usrs_ids_row, itms_ids_row,
                  loss, learning_rate, no_components, n, item_alpha, user_alpha):
    
    model = LightFM(no_components=no_components, loss=loss, learning_rate=learning_rate, 
                item_alpha=item_alpha, user_alpha=user_alpha, random_state=42,
                k=5, n=n, max_sampled=100)
    model.fit((sparse_user_item_matrix > 0) * 1,
          sample_weight=coo_matrix(user_item_mx),
          user_features=csr_matrix(user_feat_lfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lfm.values).tocsr(),
          epochs=20, num_threads=20, verbose=True)
    predictions = model.predict(user_ids=usrs_ids_row,
                            item_ids=itms_ids_row,
                            user_features=csr_matrix(user_feat_lfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lfm.values).tocsr(),
                            num_threads=10)
    data_train['score'] = predictions
    predict_result = data_train[['user_id','item_id','score']][data_train.item_id != 999999
            ].drop_duplicates().sort_values(by=['user_id','score'
            ], ascending=False).groupby('user_id')['item_id'].unique().reset_index()
    df_result_for_metrics = df.merge(predict_result, on='user_id', how='inner')
    precision = df_result_for_metrics.swifter.apply(lambda row: custom_precision(
            row['item_id'], row['actual'],k=5), axis=1).mean()
    return precision

In [ ]:
%%time

for loss, learning_rate, no_components, n, item_alpha, user_alpha in product(
    loss_func, learning_rates, no_components_list, n_list, item_alphas, user_alphas):
    res_df = result[['user_id', 'actual']]
    precision = get_precision(res_df, data_train_filtered, sparse_user_item, user_item_matrix, 
                              user_feat_lightfm, item_feat_lightfm, users_ids_row, items_ids_row,
                              loss, learning_rate, no_components, n, item_alpha, user_alpha)

    result_dict['precision_at_k'].append(precision)
    result_dict['loss'].append(loss)
    result_dict['learning_rate'].append(learning_rate)
    result_dict['no_components'].append(no_components)
    result_dict['n'].append(n)
    result_dict['item_alpha'].append(item_alpha)
    result_dict['user_alpha'].append(user_alpha)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [01:11<00:00,  3.57s/it]


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [01:07<00:00,  3.36s/it]


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [00:41<00:00,  2.06s/it]


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [01:15<00:00,  3.78s/it]


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [00:39<00:00,  2.00s/it]


...

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [00:42<00:00,  2.15s/it]


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [00:42<00:00,  2.14s/it]


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [00:41<00:00,  2.09s/it]


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 20/20 [01:05<00:00,  3.29s/it]


Epoch:  35%|██████████████████████████▌                                                 | 7/20 [00:23<00:44,  3.44s/it]

In [3]:
import json

In [ ]:
with open('./precision_bpr.json', 'w') as f:
    json.dump(result_dict, f, indent=4)

In [4]:
with open('./precision.json') as f:
    result_dict = json.load(f)

In [5]:
# Создаю датафрейм из получившихся значений
result_df = pd.DataFrame(result_dict)

In [6]:
result_df.head()

,precision_at_k,loss,learning_rate,no_components,n,item_alpha,user_alpha
0,0.124762,warp,0.1,20,10,0.2,0.1
1,0.112053,warp,0.1,20,10,0.2,0.2
2,0.115205,warp,0.1,20,10,0.2,0.3
3,0.143284,warp,0.1,20,10,0.4,0.1
4,0.143383,warp,0.1,20,10,0.4,0.2


In [7]:
result_df.loc[np.argmax(result_df.precision_at_k), :]

precision_at_k    0.146437
loss                  warp
learning_rate         0.05
no_components           40
n                       10
item_alpha             0.2
user_alpha             0.2
Name: 109, dtype: object

Удалось незначительно улучшить качество модели. Изначально $precision@k=0.1462$, после грид сёрч $precision@k=0.1464$